In [34]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import torchvision
#pip install torchvision
from torchvision import transforms, models, datasets
#https://pytorch.org/docs/stable/torchvision/index.html
import imageio
import time
import warnings
warnings.filterwarnings("ignore")
import random
import sys
import copy
import json
from PIL import Image

In [35]:
data_dir = './flower_data/'
train_dir = data_dir + 'train/'
valid_dir = data_dir + 'valid/'

In [36]:
# 01. data augmentation

In [37]:
data_transforms = {
    'train':
        transforms.Compose([
        transforms.Resize([96,96]),
        transforms.RandomRotation(45),
        transforms.CenterCrop(64),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ColorJitter(brightness=0.2, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.RandomGrayscale(p=0.025),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])       
    ]),
    'valid':
        transforms.Compose([
        transforms.Resize([64,64]),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        
        ])  
}

In [38]:
# 02. data loader

In [39]:
batch_size = 128
image_datasets = {'train': datasets.ImageFolder(train_dir, data_transforms['train']),
                 'valid': datasets.ImageFolder(valid_dir, data_transforms['valid'])
                 }



In [40]:
#image_datasets

In [41]:
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size = batch_size, shuffle=True),
    'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size = batch_size)
}

dataset_sizes = {
    'train': len(image_datasets['train']),
    'valid': len(image_datasets['valid'])
}

class_names = image_datasets['train'].classes



In [42]:
#class_names

In [43]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x2df03fdc0>,
 'valid': <torch.utils.data.dataloader.DataLoader at 0x2df03f2e0>}

In [44]:
dataset_sizes

{'train': 6552, 'valid': 818}

In [45]:
# read the actual label names

In [46]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [47]:
cat_to_name

{'21': 'fire lily',
 '3': 'canterbury bells',
 '45': 'bolero deep blue',
 '1': 'pink primrose',
 '34': 'mexican aster',
 '27': 'prince of wales feathers',
 '7': 'moon orchid',
 '16': 'globe-flower',
 '25': 'grape hyacinth',
 '26': 'corn poppy',
 '79': 'toad lily',
 '39': 'siam tulip',
 '24': 'red ginger',
 '67': 'spring crocus',
 '35': 'alpine sea holly',
 '32': 'garden phlox',
 '10': 'globe thistle',
 '6': 'tiger lily',
 '93': 'ball moss',
 '33': 'love in the mist',
 '9': 'monkshood',
 '102': 'blackberry lily',
 '14': 'spear thistle',
 '19': 'balloon flower',
 '100': 'blanket flower',
 '13': 'king protea',
 '49': 'oxeye daisy',
 '15': 'yellow iris',
 '61': 'cautleya spicata',
 '31': 'carnation',
 '64': 'silverbush',
 '68': 'bearded iris',
 '63': 'black-eyed susan',
 '69': 'windflower',
 '62': 'japanese anemone',
 '20': 'giant white arum lily',
 '38': 'great masterwort',
 '4': 'sweet pea',
 '86': 'tree mallow',
 '101': 'trumpet creeper',
 '42': 'daffodil',
 '22': 'pincushion flower',
 

In [48]:
# model build

In [49]:
model_name = 'resnet'
feature_extract = True # 使用pretrained model的特征和参数，暂时不训练，不进行gradient decent

In [50]:
train_on_gpu = torch.backends.mps.is_available()
print(train_on_gpu)

if not train_on_gpu:
    print('mps is not available, training on CPU...')
    device = torch.device('cpu')
else:
    print('mps is available, training on GPU...')
    device = torch.device('mps')
    
print(device)


True
mps is available, training on GPU...
mps


In [51]:
def set_parameter_requires_grad(model, feature_extract):
    if feature_extract:
        for param in model.parameters():
            param.requires_grad = False

            
model = models.resnet18()
model


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [52]:
def initialize_model(model, num_classes, feature_extract, use_pretrained = True):
    model = models.resnet18(pretrained=use_pretrained)
    set_parameter_requires_grad(model, feature_extract)
    
    num_features_into_fc = model.fc.in_features
    model.fc = nn.Linear(num_features_into_fc, 102) #output 102 classes
    
    input_size = 64
    
    return model, input_size
    
    

    
    

In [53]:
model_fclassifier, input_size = initialize_model(model, 102, feature_extract, use_pretrained = True)


In [ ]:
model_fclassifier = model_fclassifier.to(device)

In [ ]:
# model save
filename = 'check.pth'
# not_train all layers, only train fc layers
params_to_train = model_fclassifier.parameters()

print('Params to train/learn:')
if feature_extract:
    params_to_train = []
    for name, param in model_fclassifier.named_parameters():
        if param.requires_grad == True:
            params_to_train.append(param)
            print('\t', name)
else:
    for name, param in model_fclassifier.names_parameters():
        if param.requires_grad == True:
            print('\t', name)

    



Params to train/learn:
	 fc.weight
	 fc.bias


In [ ]:
params_to_train # the parameters need train is fc layer(weight and bias)

[Parameter containing:
 tensor([[-0.0327,  0.0168, -0.0202,  ...,  0.0415,  0.0049, -0.0282],
         [-0.0014,  0.0166,  0.0212,  ...,  0.0406,  0.0370,  0.0105],
         [-0.0409, -0.0031,  0.0380,  ...,  0.0421, -0.0127, -0.0154],
         ...,
         [-0.0043, -0.0197, -0.0171,  ...,  0.0207, -0.0398, -0.0125],
         [ 0.0123,  0.0272,  0.0323,  ..., -0.0017, -0.0306, -0.0127],
         [-0.0306,  0.0321,  0.0408,  ...,  0.0008,  0.0348,  0.0297]],
        device='mps:0', requires_grad=True),
 Parameter containing:
 tensor([-0.0110, -0.0072, -0.0154,  0.0198,  0.0162,  0.0313,  0.0273, -0.0364,
         -0.0353, -0.0251, -0.0211,  0.0083, -0.0100,  0.0410, -0.0003, -0.0195,
         -0.0156, -0.0260, -0.0107, -0.0340, -0.0293, -0.0343,  0.0267, -0.0396,
          0.0144,  0.0359, -0.0051, -0.0102,  0.0043,  0.0044, -0.0125,  0.0311,
         -0.0007, -0.0428, -0.0291,  0.0404,  0.0395,  0.0056,  0.0169, -0.0194,
          0.0439,  0.0051,  0.0392,  0.0173, -0.0146, -0.0163, 

In [ ]:
model_fclassifier

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# optimizer 

In [ ]:
optimizer = optim.Adam(params_to_train, lr = 1e-2)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma = 0.1)
# learning rate decrease to 1/10 every 10 steps
loss_func = nn.CrossEntropyLoss()

In [ ]:
def train(model, dataloaders, loss_func, optimizer, steps=30, file_name = './check.pth'):
    # time calculate
    since = time.time()
    # record the best acc
    best_acc = 0
    
    model = model.to(device)
    
    train_acc_history = []
    val_acc_history = []
    train_losses = []
    val_losses = []
    
    LRs = [optimizer.param_groups[0]['lr']]
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(steps):
        print(f'{epoch+1}/{steps}  ')
        
        # train and valid
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            loss_sum = 0
            corrects_sum = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad() # clear the gradient
                pred_outputs = model(inputs)
                loss = loss_func(pred_outputs, labels)
                _, preds = torch.max(pred_outputs, 1)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                
                loss_sum += loss.item() * inputs.size(0)
                corrects_sum += torch.sum(preds == labels).item()
            
            epoch_loss = loss_sum / len(dataloaders[phase].dataset) # average loss for each data point
            epoch_acc = corrects_sum / len(dataloaders[phase].dataset) # accuracy for one epoch
            
            time_elasped = time.time() - since
            print(f'duration: {round(time_elasped//60,2)} min {round(time_elasped%60,2)}s')
            print(f'loss: {round(epoch_loss,2)}, accuracy: {round(epoch_acc,2)}')
            
            # find the best model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                state = {
                    'state_dict': model.state_dict(),
                    'best_acc': best_acc,
                    'optimizer': optimizer.state_dict()
                
                }
                torch.save(state, filename)
            
            if phase == 'valid':
                val_acc_history.append(epoch_acc)
                val_losses.append(epoch_loss)
            
            if phase == 'train':
                train_acc_history.append(epoch_acc)
                train_losses.append(epoch_loss)
                
        current_lr = optimizer.param_groups[0]['lr']
        print(f'Optimzer learning rate: {current_lr}')
        LRs.append(current_lr)
        scheduler.step() # learning rate
    
    
    time_elapsed = time.time()-since
    print(f'Training complete in {round(time_elapsed//60,2)}min {round(time_elapsed%60,2)}s ')
    print(f'Best val accuracy: {round(best_acc,2)}')
    
    model.load_state_dict(best_model_wts)
    return model, train_acc_history, val_acc_history, train_losses, val_losses, LRs
    
    
    
    
    
    
    
    
    
    

In [ ]:
train(model_fclassifier, dataloaders, loss_func, optimizer, steps=15)

1/15  
duration: 0.0 min 23.16s
loss: 3.9, accuracy: 0.25
duration: 0.0 min 25.55s
loss: 3.64, accuracy: 0.29
Optimzer learning rate: 0.01
2/15  
duration: 0.0 min 48.52s
loss: 2.85, accuracy: 0.4
duration: 0.0 min 50.92s
loss: 3.44, accuracy: 0.31
Optimzer learning rate: 0.01
3/15  
duration: 1.0 min 13.92s
loss: 2.78, accuracy: 0.41
duration: 1.0 min 16.29s
loss: 3.55, accuracy: 0.31
Optimzer learning rate: 0.01
4/15  


In [ ]:
#model_fclassifier.state_dict()

In [ ]:
t1 = torch.tensor([1,2,3,4])
t2 = torch.tensor([1,2,1,1])
torch.sum(t1 == t2).item()

In [ ]:
t2.data

In [ ]:
s

In [ ]:
dataloaders